In [1]:
import os
import skimage
from skimage import transform
from skimage.color import rgb2gray
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory)
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f)
                      for f in os.listdir(label_directory)
                      if f.endswith(".ppm")]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return np.array(images), np.array(labels)


def plot_data(signs, labels):
    for i in range(len(signs)):
        plt.subplot(4, len(signs)/4 + 1, i+1)
        plt.axis('off')
        plt.title("Label {0}".format(labels[i]))
        plt.imshow(signs[i])
        plt.subplots_adjust(wspace=0.5)
    plt.show()

In [3]:
images, labels = load_data("belgiumTS/Training")
# display 30 random images
#randind = np.random.randint(0, len(images), 30)
#plot_data(images[randind], labels[randind])

images = np.expand_dims(rgb2gray(np.array([transform.resize(image, (50, 50)) for image in images])), axis=3)

In [4]:
# Initialize placeholders 
x_input = tf.placeholder(dtype = tf.float32, shape = [None, 50, 50, 1])
y = tf.placeholder(dtype = tf.int32, shape = [None])

# Convolution layer(s)
conv1 = tf.contrib.layers.conv2d(inputs=x_input,
                                 num_outputs=8,
                                 kernel_size=[3,3])

# Flatten the input data
images_flat = tf.contrib.layers.flatten(conv1)

# Fully-connected layer(s)
logits = tf.contrib.layers.fully_connected(images_flat, 100, tf.nn.relu)

logits = tf.contrib.layers.fully_connected(images_flat, 62, tf.nn.relu)

# Define a loss function
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                    logits = logits))
# Define an optimizer 
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

# Convert logits to label indexes
correct_pred = tf.argmax(logits, 1)

# Define an accuracy metric
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [5]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(1001):
        #print('EPOCH', i)
        _, accuracy_val, train_loss = sess.run([train_op, accuracy, loss], feed_dict={x_input: images, y: labels})
        if i % 10 == 0:
            print("Loss (", i, "): ", train_loss)
        #print('DONE WITH EPOCH')

Loss ( 0 ):  4.134216
Loss ( 10 ):  2.7979045
Loss ( 20 ):  2.2245066
Loss ( 30 ):  1.9525799
Loss ( 40 ):  1.7790805
Loss ( 50 ):  1.6547081
Loss ( 60 ):  1.576188
Loss ( 70 ):  1.5227932
Loss ( 80 ):  1.4843622
Loss ( 90 ):  1.4530287
Loss ( 100 ):  1.4294744
Loss ( 110 ):  1.4097914
Loss ( 120 ):  1.3936386
Loss ( 130 ):  1.3799673
Loss ( 140 ):  1.3684415
Loss ( 150 ):  1.3585111
Loss ( 160 ):  1.3499606
Loss ( 170 ):  1.3427266
Loss ( 180 ):  1.3363845
Loss ( 190 ):  1.3309339
Loss ( 200 ):  1.3262886
Loss ( 210 ):  1.3220586
Loss ( 220 ):  1.3184222
Loss ( 230 ):  1.3152616
Loss ( 240 ):  1.3124679
Loss ( 250 ):  1.3100126
Loss ( 260 ):  1.3080052
Loss ( 270 ):  1.30592
Loss ( 280 ):  1.3041596
Loss ( 290 ):  1.3025684
Loss ( 300 ):  1.3011526
Loss ( 310 ):  1.2998755
Loss ( 320 ):  1.2987092
Loss ( 330 ):  1.2976573
Loss ( 340 ):  1.2966928
Loss ( 350 ):  1.2958184
Loss ( 360 ):  1.2950062
Loss ( 370 ):  1.2942449
Loss ( 380 ):  1.2935634
Loss ( 390 ):  1.2929057
Loss ( 400 ):  

In [6]:
test_images, test_labels = load_data("belgiumTS/Testing")

test_images = np.expand_dims(rgb2gray(np.array([transform.resize(image, (50, 50)) for image in test_images])), axis=3)

# Run predictions against the full test set.
predicted = sess.run([correct_pred], feed_dict={x_input: test_images})[0]

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(test_labels, predicted)])
# Calculate the accuracy
accuracy = match_count / len(test_labels)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.601
